In [ ]:
# local requirements
%pip install pandas pinecone python-dotenv sentence_transformers

In [2]:
import pandas as pd

df = pd.read_csv('../dataset/src_with_desc.csv')

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# generate embeddings for generated descriptions
mass_embeddings = model.encode(list(df['generated_description']))

In [3]:
# upsert into Pinecone
from os import getenv
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
pc = Pinecone(api_key=getenv('PINECONE_API_KEY'))
index = pc.Index('nano-sous-chef')

In [21]:
def generate_upsert_format(id, embedding_result):
    return {"id": str(id), "values": embedding_result}

ENTRIES = 1024
BATCH_SIZE = 256

final_form = [generate_upsert_format(idx + 1, emb_res) for idx, emb_res in enumerate(mass_embeddings)]

batches = [final_form[i:i+BATCH_SIZE] for i in range(0, ENTRIES, BATCH_SIZE)]

In [22]:
# the actual upload
for batch in batches:
    index.upsert(vectors=batch,namespace="descriptions")

In [7]:
def fix_vector(v):
    return [float(x) for x in v]

query_text = "I really like computers and shit."
query_embedding = model.encode([query_text])[0]
# query_response = index.query(namespace="descriptions", vector=fix_vector(query_embedding), top_k=3, include_values=True)

In [ ]:
len(query_embedding)

In [ ]:
# confirming the accuracy of our retrieval
import sqlite3

db = sqlite3.connect('../cookbook.db')
cursor = db.cursor()

ids_retrieved = [match['id'] for match in query_response['matches']]
ids_retrieved = list(map(int, ids_retrieved))
print("Picks:", ids_retrieved)

for id in ids_retrieved:
    command = "SELECT name FROM information WHERE id={}".format(id)
    cursor.execute(command)
    results = cursor.fetchall()
    for row in results:
        print(row)